# Lightweight Fine-Tuning Project

* PEFT technique: [`LoRA`](https://huggingface.co/docs/peft/conceptual_guides/adapter#low-rank-adaptation-lora)
* Model: [`gpt2`](https://huggingface.co/openai-community/gpt2)
* Evaluation approach: Hugging Face's [`Trainer.evaluate`](https://huggingface.co/docs/transformers/main_classes/trainer#transformers.Trainer.evaluate)
* Fine-tuning dataset: [`imdb`](https://huggingface.co/datasets/imdb)

In [4]:
from enum import Enum

import numpy as np
import torch

from datasets import load_dataset
from IPython.display import display_markdown, Markdown
from peft import LoraConfig, TaskType, get_peft_model
from transformers import AutoModelForSequenceClassification, AutoTokenizer, DataCollatorWithPadding, GPT2Tokenizer, TrainingArguments, Trainer

I've chosen to use Hugging Face's [`evaluate`](https://huggingface.co/docs/evaluate/index) library, which may or may not be included on Udacity's workspace. We'll try to import it, and if it fails, we'll install it using pip.

In [5]:
try:
    import evaluate
except ImportError:
    import subprocess
    import sys
    subprocess.check_call([sys.executable, "-m", "pip", "install", "evaluate", "scikit-learn"])
    display_markdown(Markdown('<div class="alert alert-block alert-warning">New depencies were installed dynamically. <span style="font-weight: bold;">You should restart the kernel</span>.</div>'))

## Choosing PyTorch's device based on available backends

In [6]:
pytorch_device = torch.device("cpu")

In [7]:
if torch.cuda.is_available():
    pytorch_device = torch.device("cuda")
    display_markdown(Markdown('<div class="alert alert-block alert-success">Using CUDA backend</div>'))
elif torch.backends.mps.is_available():
    pytorch_device = torch.device("mps")
    display_markdown(Markdown('<div class="alert alert-block alert-success">Using MPS backend</div>'))
else:
    display_markdown(Markdown('<div class="alert alert-block alert-warning">Using CPU backend</div>'))

<div class="alert alert-block alert-success">Using MPS backend</div>

## Loading and Evaluating a Foundation Model

In [8]:
pretrained_model_name = "gpt2"
display(Markdown(f"### Pre-Trained Model: `{pretrained_model_name}`"))

### Pre-Trained Model: `gpt2`

In [9]:
metric_name = "accuracy"
metric = evaluate.load(metric_name)
display_markdown(Markdown(f"### Metric `{metric_name}`:\n\n```\n{metric}```"))

### Metric `accuracy`:

```
EvaluationModule(name: "accuracy", module_type: "metric", features: {'predictions': Value(dtype='int32', id=None), 'references': Value(dtype='int32', id=None)}, usage: """
Args:
    predictions (`list` of `int`): Predicted labels.
    references (`list` of `int`): Ground truth labels.
    normalize (`boolean`): If set to False, returns the number of correctly classified samples. Otherwise, returns the fraction of correctly classified samples. Defaults to True.
    sample_weight (`list` of `float`): Sample weights Defaults to None.

Returns:
    accuracy (`float` or `int`): Accuracy score. Minimum possible value is 0. Maximum possible value is 1.0, or the number of examples input, if `normalize` is set to `True`.. A higher score means higher accuracy.

Examples:

    Example 1-A simple example
        >>> accuracy_metric = evaluate.load("accuracy")
        >>> results = accuracy_metric.compute(references=[0, 1, 2, 0, 1, 2], predictions=[0, 1, 1, 2, 1, 0])
        >>> print(results)
        {'accuracy': 0.5}

    Example 2-The same as Example 1, except with `normalize` set to `False`.
        >>> accuracy_metric = evaluate.load("accuracy")
        >>> results = accuracy_metric.compute(references=[0, 1, 2, 0, 1, 2], predictions=[0, 1, 1, 2, 1, 0], normalize=False)
        >>> print(results)
        {'accuracy': 3.0}

    Example 3-The same as Example 1, except with `sample_weight` set.
        >>> accuracy_metric = evaluate.load("accuracy")
        >>> results = accuracy_metric.compute(references=[0, 1, 2, 0, 1, 2], predictions=[0, 1, 1, 2, 1, 0], sample_weight=[0.5, 2, 0.7, 0.5, 9, 0.4])
        >>> print(results)
        {'accuracy': 0.8778625954198473}
""", stored examples: 0)```

In [10]:
def compute_metrics(eval_pred):
    # Taken from https://huggingface.co/docs/evaluate/transformers_integrations#trainer
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

### Load dataset

In [11]:
train_dataset, test_dataset = load_dataset("imdb", split=("train", "test"))

In [12]:
train_dataset.to_pandas().sample(5)

,text,label
3587,"Down to Earth is about Lance Barton, a black c...",0
17791,In light of bad reviews - or car crashes - I f...,1
167,I am not so much like Love Sick as I image. Fi...,0
23049,"It's hard to say which comes out on top, James...",1
11421,I have to agree with most everyone's opinion t...,0


In [13]:
test_dataset.to_pandas().sample(5)

,text,label
11544,the worst sequel I've ever seen. really awful ...,0
20072,"Beautifully filmed, mind expanding exploration...",1
19114,"""Uzumaki"" takes place in a small Japanese rura...",1
24085,I just saw Mar Ardentro and felt that I had to...,1
19462,Oh my gosh! this was one of the best Sci-fi mo...,1


### Load tokenizer and model

In [14]:
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name)
tokenizer.pad_token = tokenizer.eos_token
display_markdown(Markdown(f"```\n{tokenizer}```"))

```
GPT2TokenizerFast(name_or_path='gpt2', vocab_size=50257, model_max_length=1024, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>', 'pad_token': '<|endoftext|>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	50256: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
}```

In [15]:
def tokenize_function(examples):
    # Taken from https://huggingface.co/docs/evaluate/transformers_integrations
    # No padding is set as suggested by https://huggingface.co/docs/transformers/tasks/sequence_classification#preprocess
    return tokenizer(examples["text"], truncation=True)

In [16]:
tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
# I'm using a small test dataset
tokenized_test_dataset = test_dataset.map(tokenize_function, batched=True).shuffle(seed=1024).select(range(20))

In [17]:
class ReviewSentiment(Enum):
    NEGATIVE = 0
    POSITIVE = 1

In [18]:
id2label = {v.value: v.name for v in ReviewSentiment}
label2id = {v.name: v.value for v in ReviewSentiment}

In [19]:
model = AutoModelForSequenceClassification.from_pretrained(
    pretrained_model_name,
    num_labels=len(id2label),
    id2label=id2label,
    label2id=label2id
).to(pytorch_device)
model.config.pad_token_id = model.config.eos_token_id

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [20]:
display(Markdown(f"```\n{model}\n```"))

```
GPT2ForSequenceClassification(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (score): Linear(in_features=768, out_features=2, bias=False)
)
```

### Evaluate pre-trained model

In [35]:
training_arguments = TrainingArguments(
    output_dir="nd608/gpt2",
    learning_rate=1e-3,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

In [36]:
trainer = Trainer(
    model=model,
    args=training_arguments,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_test_dataset,
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer, padding="max_length"),
    compute_metrics=compute_metrics
)

In [37]:
trainer.evaluate()

TypeError: GPT2ForSequenceClassification.forward() got an unexpected keyword argument 'decoder_input_ids'

## Performing Parameter-Efficient Fine-Tuning

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

In [30]:
# Taken from https://huggingface.co/docs/peft/quicktour

peft_config = LoraConfig(
    task_type=TaskType.SEQ_2_SEQ_LM,
    inference_mode=True,
    r=8,
    lora_alpha=32,
    lora_dropout=0.1
)

In [31]:
model = get_peft_model(model, peft_config)
display_markdown(Markdown(f"```\n{model}\n```"))

```
PeftModelForSeq2SeqLM(
  (base_model): LoraModel(
    (model): PeftModelForSeq2SeqLM(
      (base_model): LoraModel(
        (model): PeftModelForSeq2SeqLM(
          (base_model): LoraModel(
            (model): GPT2ForSequenceClassification(
              (transformer): GPT2Model(
                (wte): Embedding(50257, 768)
                (wpe): Embedding(1024, 768)
                (drop): Dropout(p=0.1, inplace=False)
                (h): ModuleList(
                  (0-11): 12 x GPT2Block(
                    (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
                    (attn): GPT2Attention(
                      (c_attn): lora.Linear(
                        (base_layer): Conv1D()
                        (lora_dropout): ModuleDict(
                          (default): Dropout(p=0.1, inplace=False)
                        )
                        (lora_A): ModuleDict(
                          (default): Linear(in_features=768, out_features=8, bias=False)
                        )
                        (lora_B): ModuleDict(
                          (default): Linear(in_features=8, out_features=2304, bias=False)
                        )
                        (lora_embedding_A): ParameterDict()
                        (lora_embedding_B): ParameterDict()
                      )
                      (c_proj): Conv1D()
                      (attn_dropout): Dropout(p=0.1, inplace=False)
                      (resid_dropout): Dropout(p=0.1, inplace=False)
                    )
                    (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
                    (mlp): GPT2MLP(
                      (c_fc): Conv1D()
                      (c_proj): Conv1D()
                      (act): NewGELUActivation()
                      (dropout): Dropout(p=0.1, inplace=False)
                    )
                  )
                )
                (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
              )
              (score): Linear(in_features=768, out_features=2, bias=False)
            )
          )
        )
      )
    )
  )
)
```

In [32]:
model.print_trainable_parameters()

trainable params: 0 || all params: 124,736,256 || trainable%: 0.0


In [33]:
trainer = Trainer(
    model=model,
    args=training_arguments,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_test_dataset,
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer, padding="max_length"),
    compute_metrics=compute_metrics
)

/Users/abarto/miniconda3/envs/nd608-cd13303/lib/python3.12/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


## Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.